In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
from utils import flatten_lol,remove_multi_space
import os
import pymongo
import nltk

In [18]:
nltk.download ()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
_DATASET = '/home/taindp/VINBRAIN_INTERNSHIP/explore_kb/data/dataset.csv'
dataset = pd.read_csv(_DATASET)
dataset = dataset.fillna('none_symptom')
dataset.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom
3,Fungal infection,itching,skin_rash,dischromic _patches,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom


In [3]:
list_cols = list(dataset.columns)
list_cols.pop(0)
list_merge_full = []
for idx in range(len(dataset)):
    if idx < len(dataset):
        df = dataset[idx:idx+1]
    else:
        df = dataset[idx:]
        
    list_merge_row = []
    for col in list_cols:
        symp = list(df[col])[0]
        if symp not in list_merge_row and symp != 'none_symptom':
            list_merge_row.append(symp)
    list_merge_full.append(list_merge_row)

In [4]:
dataset['Symptom'] = list_merge_full
dataset_groupby = dataset.groupby('Disease')['Symptom'].apply(list).reset_index(name='Symptom')
dataset_groupby['Symptom'] = [list(set(flatten_lol(item))) for item in list(dataset_groupby['Symptom'])]
dataset_groupby.head()

,Disease,Symptom
0,(vertigo) Paroymsal Positional Vertigo,"[ spinning_movements, vomiting, headache, n..."
1,AIDS,"[ high_fever, muscle_wasting, patches_in_thr..."
2,Acne,"[ scurring, blackheads, skin_rash, pus_fill..."
3,Alcoholic hepatitis,"[ yellowish_skin, vomiting, fluid_overload, ..."
4,Allergy,"[ chills, watering_from_eyes, shivering, co..."


In [5]:
list_sympt_norm = []
for list_sympt in list(dataset_groupby['Symptom']):
    sublist_sympt_norm = []
    for sympt in list_sympt:
        if sympt.startswith(' '):
            norm_sympt = sympt[1:]
        else:
            norm_sympt = sympt
        sublist_sympt_norm.append(norm_sympt)
    list_sympt_norm.append(sublist_sympt_norm)

In [6]:
dataset_groupby['Symptom'] = list_sympt_norm

In [10]:
list_full_sympt = list(set(flatten_lol(list(dataset_groupby['Symptom']))))
len(list_full_sympt)

131

In [19]:
list_full_sympt_split = []
for item in list_full_sympt:
    text_sympt = ' '.join(item.split('_'))
    tokens = nltk.word_tokenize(text_sympt)
    tag = nltk.pos_tag(tokens)
    print(tag)
#     list_full_sympt_split.append()
# list_full_sympt_split

[('breathlessness', 'NN')]
[('muscle', 'NN'), ('wasting', 'VBG')]
[('itching', 'VBG')]
[('obesity', 'NN')]
[('scurring', 'VBG')]
[('weight', 'NN'), ('gain', 'NN')]
[('continuous', 'JJ'), ('feel', 'NN'), ('of', 'IN'), ('urine', 'NN')]
[('cough', 'NN')]
[('knee', 'NN'), ('pain', 'NN')]
[('yellowish', 'JJ'), ('skin', 'NN')]
[('small', 'JJ'), ('dents', 'NNS'), ('in', 'IN'), ('nails', 'NNS')]
[('brittle', 'NN'), ('nails', 'NNS')]
[('hip', 'NN'), ('joint', 'NN'), ('pain', 'NN')]
[('loss', 'NN'), ('of', 'IN'), ('balance', 'NN')]
[('sweating', 'VBG')]
[('slurred', 'VBN'), ('speech', 'NN')]
[('shivering', 'VBG')]
[('abnormal', 'JJ'), ('menstruation', 'NN')]
[('headache', 'NN')]
[('inflammatory', 'NN'), ('nails', 'NNS')]
[('swollen', 'VBN'), ('blood', 'NN'), ('vessels', 'NNS')]
[('altered', 'JJ'), ('sensorium', 'NN')]
[('silver', 'NN'), ('like', 'IN'), ('dusting', 'VBG')]
[('yellow', 'JJ'), ('crust', 'NN'), ('ooze', 'NN')]
[('swollen', 'VBN'), ('legs', 'NNS')]
[('acute', 'NN'), ('liver', 'NN'), 